In [1]:
pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from fuzzywuzzy import process

# Load the datasets
df1 = pd.read_csv('/kaggle/input/pp-haxophone/merged_output.csv')  # Dataset with player features
df2 = pd.read_csv('/kaggle/input/pp-haxophone/playerfifaratings.csv')  # Dataset with player ratings

# Rename columns for uniformity
df1.rename(columns={'name': 'player_name'}, inplace=True)
df2.rename(columns={'player': 'player_name'}, inplace=True)

# Preprocess player names to remove unwanted characters and lowercase them
def preprocess_name(name):
    return re.sub(r'[^a-zA-Z0-9 ]', '', name).lower().strip()

df1['player_name'] = df1['player_name'].apply(preprocess_name)
df2['player_name'] = df2['player_name'].apply(preprocess_name)

# Function to get best match for player names (fuzzy matching)
def get_best_match(name, choices):
    match, score = process.extractOne(name, choices)
    return match if score > 90 else name  # Only return match if score is above 80

df1['player_name'] = df1['player_name'].apply(lambda x: get_best_match(x, df2['player_name'].unique()))

df2['passing'] = df2[['shortpassing', 'longpassing', 'vision']].mean(axis=1)
df2['shooting'] = df2[['finishing', 'shotpower', 'longshots', 'volleys', 'curve', 'penalties', 'composure']].mean(axis=1)
df2['defense'] = df2[['interceptions', 'defensiveawareness', 'standingtackle', 'slidingtackle', 'aggression']].mean(axis=1)
df2['keeping_skills'] = df2[['gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']].mean(axis=1)

df2 = df2[['player_name', 'team', 'league', 'fifa_edition', 'update', 'overallrating', 'potential',
           'passing', 'shooting', 'defense', 'keeping_skills']]

merged_df = df1.merge(df2, left_on='player_name', right_on='player_name', how='inner')

print(len(merged_df))

# Save the merged dataset
merged_df.to_csv('merged_dataset.csv', index=False)

print("Merged dataset saved successfully.")


2048
Merged dataset saved successfully.


In [3]:
df2.to_csv('all_points.csv', index=False)
df2

,player_name,team,league,fifa_edition,update,overallrating,potential,passing,shooting,defense,keeping_skills
0,stefan ortega,Manchester City,ENG-Premier League,FC 25,"Feb 5, 2025",81,81,58.666667,30.714286,22.4,81.4
1,matheus luiz nunes,Manchester City,ENG-Premier League,FC 25,"Feb 5, 2025",78,80,78.666667,70.142857,75.4,12.0
2,rben santos gato alves dias,Manchester City,ENG-Premier League,FC 25,"Feb 5, 2025",87,88,76.666667,48.285714,87.4,9.4
3,manuel akanji,Manchester City,ENG-Premier League,FC 25,"Feb 5, 2025",83,83,75.666667,54.285714,83.6,9.4
4,joko gvardiol,Manchester City,ENG-Premier League,FC 25,"Feb 5, 2025",84,87,75.666667,71.285714,84.4,7.8
...,...,...,...,...,...,...,...,...,...,...,...
2781,enrique prez muoz,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",73,74,75.666667,61.857143,59.8,12.2
2782,bjarki steinn bjarkason,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",65,69,64.000000,55.857143,48.6,10.4
2783,richie sagrado,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",66,78,55.000000,41.000000,60.2,9.4
2784,saad el haddad,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",54,70,52.666667,48.714286,36.4,10.8


In [4]:
merged_df

,player_name,nation,pos,born,Club,league_x,ID,Tackles_defense,Challenges_defense,Blocks_defense,...,team,league_y,fifa_edition,update,overallrating,potential,passing,shooting,defense,keeping_skills
0,marcus bettinelli,ENG,GK,1992.0,Fulham,ENG-Premier League,1819.0,0.000000,0.000000,0.000000,...,Chelsea,ENG-Premier League,FC 25,"Feb 5, 2025",70,70,38.000000,24.285714,19.6,70.0
1,tom heaton,ENG,GK,1986.0,Aston Villa,ENG-Premier League,1920.0,1.066667,14.166667,0.000000,...,Manchester United,ENG-Premier League,FC 25,"Feb 5, 2025",70,70,43.666667,27.714286,19.8,70.4
2,david raya martin,ENG,GK,1986.0,West Ham,ENG-Premier League,1920.0,0.000000,0.000000,0.000000,...,Arsenal,ENG-Premier League,FC 25,"Feb 5, 2025",85,86,65.666667,32.857143,19.0,85.0
3,kortney hause,ENG,DF,1995.0,Aston Villa,ENG-Premier League,2021.0,4.733333,7.941667,7.777778,...,Aston Villa,ENG-Premier League,FC 25,"Feb 5, 2025",70,70,61.333333,42.428571,70.4,13.8
4,scott carson,ENG,GK,1985.0,Manchester City,ENG-Premier League,2021.0,0.000000,0.000000,0.000000,...,Manchester City,ENG-Premier League,FC 25,"Feb 5, 2025",63,63,28.666667,24.857143,17.4,62.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,michael svoboda,AUT,DF,1998.0,Venezia,ITA-Serie A,2425.0,13.200000,20.025000,12.666667,...,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",69,72,58.333333,44.142857,67.2,9.0
2044,mikael egill ellertsson,ISL,"MF,DF",2002.0,Venezia,ITA-Serie A,2425.0,12.000000,20.583333,8.888889,...,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",66,76,67.000000,66.142857,60.0,10.8
2045,richie sagrado,BEL,DF,2004.0,Venezia,ITA-Serie A,2425.0,1.200000,25.500000,0.666667,...,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",66,78,55.000000,41.000000,60.2,9.4
2046,ridgeciano haps,SUR,DF,1993.0,Venezia,ITA-Serie A,2425.0,7.200000,21.541667,6.888889,...,Venezia,ITA-Serie A,FC 25,"Feb 5, 2025",69,69,61.000000,61.142857,65.0,11.0


In [5]:
column_name = 'pos' 
value_counts = merged_df[column_name].value_counts()
print(value_counts)

pos
DF       582
MF       447
FW       265
FW,MF    214
GK       192
MF,FW    162
DF,MF     83
MF,DF     47
DF,FW     40
FW,DF     16
Name: count, dtype: int64
